In [1]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, Merge
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

Using TensorFlow backend.
/Users/DuaaTashkandi/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#parameters
maxlen = 30
labels = 2

In [3]:
input = pd.read_csv("data_new/merged.csv",header=None)
input.columns = ['first', 'last','b_or_n']

# remove encode 
input['first'] = input['first'].str[2:-1]
input['last'] = input['last'].str[2:-1]

input['firstlen']= [len(str(i)) for i in input['first']]
input['lastlen'] = [len(str(i)) for i in input['last']]
input1 = input[(input['firstlen'] >= 2) & (input['lastlen'] >= 2)]

In [4]:
firsts = input['first']
lasts = input['last']
labels = input['b_or_n']

vocab = set(' '.join([str(i) for i in firsts]))
vocab = set(' '.join([str(i) for i in lasts]))

vocab.add('END')
len_vocab = len(vocab)

In [5]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [6]:
#train test split 
msk = np.random.rand(len(input1)) < 0.8

train = input1[msk]
test = input1[~msk]

In [7]:
def set_flag(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [8]:
# Truncating and padding training data
train_X = []
train_Y = []
train_Z = []

trunc_train_first = [str(i)[0:maxlen] for i in train['first']]
trunc_train_last = [str(i)[0:maxlen] for i in train['last']]

for i in trunc_train_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)

for i in trunc_train_last:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_Y.append(tmp)
    
for i in train['b_or_n']:
    if i == 1:
        train_Z.append([1,0])
    else:
        train_Z.append([0,1])

In [9]:
np.asarray(train_X).shape

(48013, 30, 39)

In [10]:
np.asarray(train_Y).shape

(48013, 30, 39)

In [11]:
np.asarray(train_Z).shape

(48013, 2)

In [12]:
#build the model: 2 stacked LSTM
print('Building LSTM model')

left_branch = Sequential()
left_branch.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
right_branch = Sequential()
right_branch.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))

model = Sequential()
model.add(Merge([left_branch, right_branch], mode='concat'))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))

# Softmax activation function
model.add(Activation('softmax'))

# Cross-entropy loss, metric is accuracy
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Building LSTM model


/Users/DuaaTashkandi/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # Remove the CWD from sys.path while we load stuff.


In [13]:
# Truncating and padding test data

test_X = []
test_Y = []
test_Z = []

trunc_test_first = [str(i)[0:maxlen] for i in test['first']]
trunc_test_last = [str(i)[0:maxlen] for i in test['last']]

for i in trunc_test_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)

for i in trunc_test_last:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_Y.append(tmp)
    
for i in test['b_or_n']:
    if i == 1:
        test_Z.append([1,0])
    else:
        test_Z.append([0,1]) 

In [14]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)
print(np.asarray(test_Z).shape)

(11941, 30, 39)
(11941, 30, 39)
(11941, 2)


In [15]:
batch_size = 1000
model.fit([np.asarray(train_X), np.asarray(train_Y)], train_Z, batch_size=batch_size, nb_epoch=50, validation_data=([np.asarray(test_X), np.asarray(test_Y)], test_Z))

/Users/DuaaTashkandi/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 48013 samples, validate on 11941 samples
Epoch 1/50
48013/48013 [==============================] - 1474s 31ms/step - loss: 0.6589 - acc: 0.5925 - val_loss: 0.5259 - val_acc: 0.7562
Epoch 2/50
48013/48013 [==============================] - 1407s 29ms/step - loss: 0.4934 - acc: 0.7720 - val_loss: 0.5144 - val_acc: 0.7621
Epoch 3/50
48013/48013 [==============================] - 1473s 31ms/step - loss: 0.4585 - acc: 0.7941 - val_loss: 0.5232 - val_acc: 0.7586
Epoch 4/50
48013/48013 [==============================] - 1442s 30ms/step - loss: 0.4429 - acc: 0.8017 - val_loss: 0.4991 - val_acc: 0.7864
Epoch 5/50
48013/48013 [==============================] - 1495s 31ms/step - loss: 0.4213 - acc: 0.8171 - val_loss: 0.4070 - val_acc: 0.8232
Epoch 6/50
48013/48013 [==============================] - 1493s 31ms/step - loss: 0.3961 - acc: 0.8288 - val_loss: 0.4094 - val_acc: 0.8248
Epoch 7/50
48013/48013 [==============================] - 1508s 31ms/step - loss: 0.3776 - acc: 0.8404 - val_l

In [17]:
score, acc = model.evaluate([np.asarray(test_X), np.asarray(test_Y)], test_Z)
print('Test score:', score)
print('Test accuracy:', acc)

11941/11941 [==============================] - 141s 12ms/step
Test score: 0.292373455935
Test accuracy: 0.900510844993


In [18]:
#save our model and data
model.save_weights('model', overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [19]:
evals = model.predict([np.asarray(test_X), np.asarray(test_Y)])
prob_m = [i[0] for i in evals]

In [20]:
out = pd.DataFrame(prob_m)
out['first'] = test['first'].reset_index()['first']
out['last'] = test['last'].reset_index()['last']
out['b_or_n'] = test['b_or_n'].reset_index()['b_or_n']

In [21]:
out.head(10)
out.columns = ['prob_b','first', 'last', 'actual']
out.head(10)
out.to_csv("pred_out.csv")

In [45]:
# small test. first 3 names are spanish and last 3 names are Brazilian 
first = ["manuel","luis","ezequil","michel","luiz","alvaro"]
last = ["garcia","sanchez","gomez","caetano","guilherme","lime"]
X = []
Y = []
trunc_first = [i[0:maxlen] for i in first]
trunc_last = [i[0:maxlen] for i in last]
for i in trunc_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
for i in trunc_last:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    Y.append(tmp)
    
pred = model.predict([np.asarray(X), np.asarray(Y)])
pred

array([[ 0.41819027,  0.58180976],
       [ 0.36226737,  0.63773263],
       [ 0.61281317,  0.38718686],
       [ 0.85553694,  0.14446311],
       [ 0.99840802,  0.00159197],
       [ 0.98669618,  0.01330382]], dtype=float32)